Goal, create an interactive plot (via Plotly) to show each individual geospacial dataset we used

- Places
- Outages
- Substation Locations

In [ ]:
import sys
import pandas as pd
import folium

# Allow importing from parent directory by temporarily moving the CWD up one level
# Very hacky, but there literally isn't a simpler way (in Jupyter)
sys.path.append("..")
from common import get_dataframe_from_pipeline

outages = get_dataframe_from_pipeline("../pipeline/3.csv.gz")
# Drop the path back down after import
sys.path.pop()

substations = pd.read_csv("../_datasets/BCSubstationLocations.csv")
populatedPlaces = pd.read_csv("../_datasets/BCPopulatedPlaces.csv")

In [ ]:
import folium.plugins


m = folium.Map(
  tiles="OpenStreetMap",
  location=(outages["outageLatitude"].mean(), outages["outageLongitude"].mean()),
  zoom_start=6,
)

outages["duration"] = outages["dateOn"] - outages["dateOff"]

# Make all the clusters the same colour
icon_create_function = """
    function(cluster) {
    var childCount = cluster.getChildCount(); 
    var c = ' marker-cluster-small';

    return new L.DivIcon({ html: '<div><span>' + childCount + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) });
    }
    """

outageGroup = folium.plugins.MarkerCluster(
  name="Outages", show=True, icon_create_function=icon_create_function
).add_to(m)
outages.apply(
  lambda x: folium.Marker(
    location=[x["outageLatitude"], x["outageLongitude"]],
    popup=folium.Popup(
      f"Id: {x['outageId']}<br>Duration: {x['duration']}",
    ),
    tooltip=f"Outage {x['outageId']}",
    icon=folium.Icon(color="red", icon="bolt", prefix="fa"),
  ).add_to(outageGroup),
  axis="columns",
)

substationGroup = folium.plugins.MarkerCluster(
  name="Substations", show=True, icon_create_function=icon_create_function
).add_to(m)
substations.apply(
  lambda x: folium.Marker(
    location=[x["latitude"], x["longitude"]],
    popup=folium.Popup(
      f"Name: {x['name']}<br>Id: {x['id']}",
    ),
    tooltip=f"Substation {x['name']}",
    icon=folium.Icon(color="blue", icon="tint"),
  ).add_to(substationGroup),
  axis="columns",
)


def isReserve(x):
  # Use a different icon for Indigenous Reserves because
  # there are so many of them and they are often very small
  return (x["generic term"] == "Indian Reserve") or (
    x["generic term"] == "First Nation Administrative Location"
  )


populatedPlaceGroup = folium.plugins.MarkerCluster(
  name="Populated Places", show=True, icon_create_function=icon_create_function
).add_to(m)

populatedPlaces.apply(
  lambda x: folium.Marker(
    location=[x["latitude"], x["longitude"]],
    popup=folium.Popup(
      f"Name: {x['geographical name']}<br>Population: {x['population, 2021']}",
    ),
    tooltip=f"Populated Place {x['geographical name']}",
    icon=folium.Icon(
      color="green", icon="tree-city" if isReserve(x) else "city", prefix="fa"
    ),
  ).add_to(populatedPlaceGroup),
  axis="columns",
)

folium.LayerControl().add_to(m)

m

In [ ]:
import os

# Save the map
# Make directory if it doesn't exist
if not os.path.exists("html"):
  os.makedirs("html")

m.save("html/datasetVisualization.html")